# Flooding Maps Denmark

In [1]:
import pandas as pd
from tqdm import tqdm
import os 

import rasterio
from owslib.wms import WebMapService

from climatemapping.maps import check_layer_number,download_map
from climatemapping.maps import save_merged_map

## Download data

**Right now this is an issue because we do not have access to the data at Danske Bank due to firewall protection. I've done it in my personal computer and then download it from my github account**

The code is available here in DownloadData.ipynb

https://notebooks.gesis.org/binder/v2/gh/VeraPatricio/floodingDK/master

## Data from 2019

This is for only one scenario (1/1000 years floods) but includes all Denmark

https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer/WMSServer?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities
 
There are several layers in this one, but the two that covers all of Denmark for an extreme flooding is

-          Fare fra hav (ekstrem hændelse)
-          Fare fra vandløb (1000 år)

In [2]:
url_2019 = 'https://gis.nst.dk/server/services/ekstern/OSD_2016_2021_Plantrin1_Endelige_data/MapServer'
wms_2019= WebMapService(url_2019+'/WMSServer')

for layer_nb in wms_2019.contents:
    print('%s %40s %s'%(layer_nb,wms_2019[layer_nb].title,wms_2019[layer_nb].crsOptions))

0                         Hovedvandoplande ['EPSG:4326', 'EPSG:25832']
1                     Vandområdedistrekter ['EPSG:4326', 'EPSG:25832']
2                           Kommunegrænser ['EPSG:4326', 'EPSG:25832']
3      Potentielt Forurenende Virksomheder ['EPSG:4326', 'EPSG:25832']
4                      Økonomisk Aktivitet ['EPSG:4326', 'EPSG:25832']
5                                Kulturarv ['EPSG:4326', 'EPSG:25832']
6                    Kritisk Infrastruktur ['EPSG:4326', 'EPSG:25832']
7                            Infrastruktur ['EPSG:4326', 'EPSG:25832']
8                                Beredskab ['EPSG:4326', 'EPSG:25832']
9                               Befolkning ['EPSG:4326', 'EPSG:25832']
10                          Arealanvendelse ['EPSG:4326', 'EPSG:25832']
11               Fare fra vandløb (1000 år) ['EPSG:4326', 'EPSG:25832']
12          Fare fra hav (ekstrem hændelse) ['EPSG:4326', 'EPSG:25832']
13                         Samlet sårbarhed ['EPSG:4326', 'EPSG:25832']
14

Download Data

In [3]:
file_name_list = []
bbox_list = []

dir_path = 'FloodingMaps/FloodingDK2019/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

file_name,bbox = download_map(url_2019,layer='11',size=(4000,4000),crs='EPSG:4326',output_dir=dir_path)
file_name_list.append(file_name)
bbox_list.append(bbox)
                                
file_name,bbox = download_map(url_2019,layer='12',size=(4000,4000),crs='EPSG:4326',output_dir=dir_path)
file_name_list.append(file_name)
bbox_list.append(bbox)   

Saved FloodingMaps/FloodingDK2019/Fare fra vandløb (1000 år).png
Saved FloodingMaps/FloodingDK2019/Fare fra hav (ekstrem hændelse).png


## Merge two maps (costal and river flooding) into one

In [4]:
bbox_merged = save_merged_map(['FloodingMaps/FloodingDK2019/Fare fra hav (ekstrem hændelse)_georef.png',
                               'FloodingMaps/FloodingDK2019/Fare fra vandløb (1000 år)_georef.png'],
                                resolution=None, # keeps the original
                                out_file='FloodingMaps/FloodingDK2019/Fare_Hav_og_Vanlob_1000ar')
file_name_list.append('FloodingMaps/FloodingDK2019/Fare_Hav_og_Vanlob_1000ar.png')
bbox_list.append(bbox_merged)

Save file with files names and bbox

In [5]:
df_png = pd.DataFrame({'scenario':['2019']*3,
                       'file':file_name_list,
                       'geo_zone':['DK']*3,
                       'bbox':bbox_list})
df_png.to_csv('map_list_DK_2019')
df_png.tail(100)

,scenario,file,geo_zone,bbox
0,2019,FloodingMaps/FloodingDK2019/Fare fra vandløb (...,DK,"(8.019848, 54.445098, 15.562142, 57.653192)"
1,2019,FloodingMaps/FloodingDK2019/Fare fra hav (ekst...,DK,"(8.008285, 54.435388, 15.599735, 57.769964)"
2,2019,FloodingMaps/FloodingDK2019/Fare_Hav_og_Vanlob...,DK,"(8.008285, 54.435388, 15.599735, 57.769964)"


## Data from 2020

The data is available in this external server: https://gis.nst.dk/server/rest/services/ekstern

and a list of the different maps available is here:

In [6]:
df_flood_maps = pd.read_csv('DK_maps_server.csv')
df_flood_maps.head(20)

,zone,url
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...
6,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...
7,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...
8,Koege,https://gis.nst.dk/server/services/ekstern/Koe...
9,Nyborg,https://gis.nst.dk/server/services/ekstern/Nyb...


Each map has several layers. We can check one of them, to find the flooding maps we are interested in:

**breaks down at the office**

In [7]:
wms = WebMapService('https://gis.nst.dk/server/services/ekstern/Kobenhavn_NS_Plantrin2/MapServer/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

1                         Risikoområde_KHS ['EPSG:4326', 'EPSG:25832']
4 Kulturarv_Seværdige_Fortidsminder_punkter_KHS ['EPSG:4326', 'EPSG:25832']
5 Kulturarv_Seværdige_Fortidsminder_areal_KHS ['EPSG:4326', 'EPSG:25832']
6                     Kulturarv_Kirker_KHS ['EPSG:4326', 'EPSG:25832']
7          Kulturarv_Fredede_Bygninger_KHS ['EPSG:4326', 'EPSG:25832']
8      Kulturarv_Fortidsminder_punkter_KHS ['EPSG:4326', 'EPSG:25832']
9        Kulturarv_Fortidsminder_areal_KHS ['EPSG:4326', 'EPSG:25832']
10                   Bes_sten_jorddiger_KHS ['EPSG:4326', 'EPSG:25832']
11                   Begravelsesomraade_KHS ['EPSG:4326', 'EPSG:25832']
13                     Vandvaerksboring_KHS ['EPSG:4326', 'EPSG:25832']
14 Miljoeinteresser_Vandforsyningsboring_KHS ['EPSG:4326', 'EPSG:25832']
15          Miljoeinteresser_Natura2000_KHS ['EPSG:4326', 'EPSG:25832']
16     Miljoeinteresser_Beskyttet_natur_KHS ['EPSG:4326', 'EPSG:25832']
17 Miljoeinteresser_Aktiv_vandindvinding_KHS ['EPSG:4326', 'EP

We're going to use the layers called 'Udbredelse' something something. We need to check what is the number of this layer in each of the maps (i.e. the map of each zone)

In [8]:
scenarios = ['MTEXT_2115',
             'MTEXT_2019',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019',
             'MT50_2115',
             'MT50_2065',
             'MT50_2019',
            ]

for scenario in scenarios:
    print(scenario)
    df_flood_maps['Udbredelse_'+scenario] = df_flood_maps.url.apply(lambda x: check_layer_number(x,'Udbredelse_\w+_'+scenario))

MTEXT_2115
MTEXT_2019
MT100_2115
MT100_2065
MT100_2019
MT20_2019
MT50_2115
MT50_2065
MT50_2019


In [9]:
file_name_list =  []
bbox_list = []
scenario_list = []
geo_zone_list = []

for scenario in tqdm(scenarios):
    
    dir_path = 'FloodingMaps/Udbredelse_'+scenario+'/'
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    for idx,row in df_flood_maps.iterrows():
        if len(row['Udbredelse_'+scenario]) == 0:
            print('Scenario for ',row['Udbredelse_'+scenario],'does not exist')
            continue
        else:
            # there can be more than one layer (eg. cph north and south)
            for layer in row['Udbredelse_'+scenario]:
                file_name,bbox = download_map(row['url'],
                                               layer=layer,
                                               size=(4000,4000),
                                               crs='EPSG:4326',
                                               output_dir=dir_path)

                file_name_list.append(file_name)
                geo_zone_list.append(file_name.split(dir_path)[1].split(scenario)[0].split('Udbredelse_')[1].strip('_'))
                bbox_list.append(bbox)
                scenario_list.append(scenario)                

  0%|          | 0/9 [00:00<?, ?it/s]

Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_AAB_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_ESB_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_FRE_MTEXT_2115.png
Scenario for  [] does not exist
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_JUE_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_KHS_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_KHN_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_KOL_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_KOR_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_KBU_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_NYB_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_ODE_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_RAF_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_SKB_MTEXT_2115.png
Saved FloodingMaps/Udbredelse_MTEXT_2115/Udbr

 11%|█         | 1/9 [00:58<07:49, 58.70s/it]

Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_AAB_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_ESB_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_FRE_MTEXT_2019.png
Scenario for  [] does not exist
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_JUE_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_KHS_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_KHN_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_KOL_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_KOR_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_KBU_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_NYB_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_ODE_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_RAF_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbredelse_SKB_MTEXT_2019.png
Saved FloodingMaps/Udbredelse_MTEXT_2019/Udbr

 22%|██▏       | 2/9 [01:57<06:51, 58.78s/it]

...

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Saved FloodingMaps/Udbredelse_MT50_2065/Udbredelse_HOL_MT50_2065.png


 89%|████████▉ | 8/9 [05:28<00:27, 27.30s/it]

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Saved FloodingMaps/Udbredelse_MT50_2019/Udbredelse_HOL_MT50_2019.png


100%|██████████| 9/9 [05:31<00:00, 36.79s/it]

Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist
Scenario for  [] does not exist


Save the name, bbox and scenario in a file

In [24]:
df_png = pd.DataFrame({'scenario':scenario_list,
                       'file':file_name_list,
                       'bbox':bbox_list,
                       'geo_zone':geo_zone_list
                      })
df_png.to_csv('map_list_DK_2020')
df_png.tail(100)

,scenario,file,bbox,geo_zone
4,MTEXT_2115,FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_...,"(12.43739, 55.5522, 12.693495, 55.714904)",KHS
5,MTEXT_2115,FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_...,"(12.487415, 55.552264, 12.694708, 55.726316)",KHN
6,MTEXT_2115,FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_...,"(9.377683, 55.47553, 9.553077, 55.510132)",KOL
7,MTEXT_2115,FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_...,"(11.090517, 55.307539, 11.217802, 55.363792)",KOR
8,MTEXT_2115,FloodingMaps/Udbredelse_MTEXT_2115/Udbredelse_...,"(12.120189, 55.410787, 12.468536, 55.639661)",KBU
...,...,...,...,...
99,MT20_2019,FloodingMaps/Udbredelse_MT20_2019/Udbredelse_V...,"(9.542685, 55.690325, 9.668523, 55.710755)",VEJ
100,MT20_2019,FloodingMaps/Udbredelse_MT20_2019/Udbredelse_V...,"(11.840523, 54.97512, 11.936832, 55.019609)",VOR
101,MT50_2115,FloodingMaps/Udbredelse_MT50_2115/Udbredelse_H...,"(8.590139, 56.34769, 8.638287, 56.360013)",HOL
102,MT50_2065,FloodingMaps/Udbredelse_MT50_2065/Udbredelse_H...,"(8.590139, 56.34769, 8.638287, 56.360013)",HOL


### Merging files

Some of the maps overlap, and this is a bit problematic further down, because a property is going to be assigned to two zones. The easiest way is to merge these overlapping maps into only one.

In [11]:
# not including the MT50
scenarios = ['MTEXT_2019',
             'MTEXT_2115',
             'MT100_2115',
             'MT100_2065',
             'MT100_2019',
             'MT20_2019']

In [25]:
def update_map_list(map_list,scenario,drop_zone,new_zone,new_file,new_bbox):    
    df_map_list = pd.read_csv(map_list,index_col=0)
    for z in drop_zone:
        idx = df_map_list[(df_map_list.geo_zone == z) &
                            (df_map_list.scenario == scenario)
                           ].index
        df_map_list.drop(idx,inplace=True)
    new_row = {'scenario':scenario,
               'file':new_file,
               'bbox':new_bbox,
               'geo_zone':new_zone
              }
    df_map_list.append(new_row,ignore_index=True)
    df_map_list.to_csv(map_list)

**Copenhagen region (also including Køge)**

In [26]:
for scenario in scenarios:
    outfile = 'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHSN_'+scenario
    bbox = save_merged_map(
                ['FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHS_'+scenario+'_georef.png',
                 'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KHN_'+scenario+'_georef.png',
                 'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_KBU_'+scenario+'_georef.png'
                ],
                resolution=None, # keeps the original
                out_file=outfile)

    update_map_list('map_list_DK_2020',
                    drop_zone=['KHN','KHS','KBU'],
                    scenario=scenario,
                    new_zone='KHSN',
                    new_file=outfile,
                    new_bbox=(bbox.left,bbox.bottom,bbox.right,bbox.top)    
                   )

**Loland**

Loland (SLO) is split in the extreme scenarios. To simplify things, we'll merge them and call it also SLO 

In [27]:
for scenario in ['MTEXT_2019','MTEXT_2115']:
    outfile = 'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_SLO_'+scenario
    bbox = save_merged_map(
                ['FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_SYD_'+scenario+'_georef.png',
                 'FloodingMaps/Udbredelse_'+scenario+'/Udbredelse_NAK_'+scenario+'_georef.png'],
                resolution=None,
                out_file=outfile)
    
    update_map_list('map_list_DK_2020',
                    drop_zone=['SYD','NAK'],
                    scenario=scenario,
                    new_zone='SLO',
                    new_file=outfile,
                    new_bbox=(bbox.left,bbox.bottom,bbox.right,bbox.top)    
                   )

**Holstebro region**

In [28]:
for year in ['2115','2019']:
    outfile='FloodingMaps/Udbredelse_MT50_'+year+'/Udbredelse_HOL_MT150_'+year
    transform = save_merged_map(
                    ['FloodingMaps/Udbredelse_MT100_'+year+'/Udbredelse_HOL_MT100_'+year+'_georef.png',
                     'FloodingMaps/Udbredelse_MT50_'+year+'/Udbredelse_HOL_MT50_'+year+'_georef.png'],
                    resolution=None, # keeps the original
                    out_file=outfile)
    
    new_row = {'scenario':'MT150',
               'file':outfile,
               'bbox':(transform.left,transform.bottom,transform.right,transform.top)   ,
               'geo_zone':'HOL'
              }
    new_list=pd.read_csv('map_list_DK_2020',index_col=0).append(new_row,ignore_index=True)
    new_list.to_csv('map_list_DK_2020')

## Data for Sweden

Maps are available here: https://gisapp.msb.se/Apps/oversvamningsportal/hemta-data.html

We'll be using the 200-årsflöde (Klimatanpassat flöde för slutet av seklet) and 100-årsflöde (Klimatanpassat flöde för slutet av seklet)

OrderedDict([('0', <owslib.map.wms111.ContentMetadata at 0x11268e400>),
             ('1', <owslib.map.wms111.ContentMetadata at 0x11200c940>),
             ('2', <owslib.map.wms111.ContentMetadata at 0x111938160>),
             ('3', <owslib.map.wms111.ContentMetadata at 0x111938828>),
             ('4', <owslib.map.wms111.ContentMetadata at 0x111938cf8>),
             ('5', <owslib.map.wms111.ContentMetadata at 0x111938e10>),
             ('6', <owslib.map.wms111.ContentMetadata at 0x111938f28>),
             ('7', <owslib.map.wms111.ContentMetadata at 0x111941080>),
             ('8', <owslib.map.wms111.ContentMetadata at 0x111941198>),
             ('9', <owslib.map.wms111.ContentMetadata at 0x1119412b0>),
             ('10', <owslib.map.wms111.ContentMetadata at 0x1119413c8>),
             ('11', <owslib.map.wms111.ContentMetadata at 0x111941518>),
             ('12', <owslib.map.wms111.ContentMetadata at 0x1119416a0>),
             ('13', <owslib.map.wms111.ContentMetadata at 0x1

In [30]:
url_swe = 'https://gisapp.msb.se/arcgis/services/Oversvamningskarteringar/karteringar/MapServer'
wms_swe= WebMapService(url_swe+'/WMSServer')

for layer_nb in wms_swe.contents:
    print('%s %40s %s'%(layer_nb,wms_swe[layer_nb].title,wms_swe[layer_nb].crsOptions))

0      Översiktliga, beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
1               Översiktliga, 100-årsflöde ['EPSG:4326', 'EPSG:3006']
2  Tvärsektioner, översiktliga karteringar ['EPSG:4326', 'EPSG:3006']
3         Torne älv, beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
4                  Torne älv, 250-årsflöde ['EPSG:4326', 'EPSG:3006']
5                  Torne älv, 100-årsflöde ['EPSG:4326', 'EPSG:3006']
6                 Torne älv, tvärsektioner ['EPSG:4326', 'EPSG:3006']
7 Göta älv, 1400 m³/s (Motsvarande beräknat högsta flöde) ['EPSG:4326', 'EPSG:3006']
8 Göta älv, 1200 m³/s (Motsvarande 200-årsflöde) ['EPSG:4326', 'EPSG:3006']
9 Göta älv, 1030 m³/s (Motsvarande 100-årsflöde) ['EPSG:4326', 'EPSG:3006']
10                  Göta älv, tvärsektioner ['EPSG:4326', 'EPSG:3006']
11                    Beräknat högsta flöde ['EPSG:4326', 'EPSG:3006']
12 200-årsflöde (Klimatanpassat flöde för slutet av seklet) ['EPSG:4326', 'EPSG:3006']
13 100-årsflöde (Klimatanpassat flöde för sl

In [32]:
# Create path
dir_path = 'FloodingMaps/FloodingSWE2020/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# save info
file_scenario_list = []
file_name_list = []
bbox_list = []

file_name,bbox = download_map(url_swe,layer='12',size=(4000,4000),crs='EPSG:4326',output_dir='FloodingMaps/FloodingSWE2020/')
file_scenario_list.append('200-years')
file_name_list.append(file_name)
bbox_list.append(bbox)
                                
file_name,bbox = download_map(url_swe,layer='13',size=(4000,4000),crs='EPSG:4326',output_dir='FloodingMaps/FloodingSWE2020/')
file_scenario_list.append('100-years')
file_name_list.append(file_name)
bbox_list.append(bbox)

df_png = pd.DataFrame({'scenario':file_scenario_list,
                       'file':file_name_list,
                       'bbox':bbox_list,
                       'geo_zone':['SWE']*2})
df_png.to_csv('map_list_SWE_2020')
df_png.tail(100)

Saved FloodingMaps/FloodingSWE2020/200-årsflöde (Klimatanpassat flöde för slutet av seklet).png
Saved FloodingMaps/FloodingSWE2020/100-årsflöde (Klimatanpassat flöde för slutet av seklet).png


,scenario,file,bbox,geo_zone
0,200-years,FloodingMaps/FloodingSWE2020/200-årsflöde (Kli...,"(10.424994, 55.472992, 23.711993, 67.43329)",SWE
1,100-years,FloodingMaps/FloodingSWE2020/100-årsflöde (Kli...,"(10.255073, 55.472992, 24.032033, 68.288574)",SWE
